In [1]:
from database.mainstbets import MainStBets
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timedelta
import matplotlib.pyplot as plt

In [2]:
market = MainStBets()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [4]:
sp500.columns

Index(['_id', 'Symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK',
       'Founded'],
      dtype='object')

In [5]:
prices["date"] = pd.to_datetime(prices["date"])

In [6]:
prices.columns

Index(['_id', 'date', 'close', 'high', 'low', 'open', 'volume', 'adjClose',
       'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor',
       'ticker'],
      dtype='object')

In [7]:
final = []
for ticker in tqdm(sp500["Symbol"]):
    ticker_data = prices[prices["ticker"]==ticker]
    ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
    ticker_data["gain"] = (ticker_data["adjClose"] - ticker_data["rolling"]) / ticker_data["rolling"]
    final.append(ticker_data)

  0%|                                                                                                                                                    | 0/505 [00:00<?, ?it/s]C:\Users\ericj\AppData\Local\Temp/ipykernel_17020/1279899860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["rolling"] = ticker_data["adjClose"].rolling(window=100).mean()
C:\Users\ericj\AppData\Local\Temp/ipykernel_17020/1279899860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data["gain"] = (ticker_dat

In [8]:
sim = pd.concat(final)
sim.dropna(inplace=True)

In [9]:
sim["year"] = [x.year for x in sim["date"]]

In [10]:
today = sim[sim["date"]==sim["date"].max()][["date","adjClose","ticker","gain","rolling"]].sort_values("gain").merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")

In [11]:
recent = today[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [12]:
recent

,date,GICS Sector,Security,ticker,rolling,adjClose,gain
0,2021-08-25 00:00:00+00:00,Consumer Discretionary,Las Vegas Sands,LVS,52.503300,43.24,-0.176433
1,2021-08-25 00:00:00+00:00,Information Technology,IPG Photonics,IPGP,203.733900,172.15,-0.155025
2,2021-08-25 00:00:00+00:00,Materials,FMC Corporation,FMC,109.753948,92.90,-0.153561
3,2021-08-25 00:00:00+00:00,Information Technology,Global Payments,GPN,193.063822,165.25,-0.144065
4,2021-08-25 00:00:00+00:00,Consumer Discretionary,Wynn Resorts,WYNN,117.614100,100.84,-0.142620
...,...,...,...,...,...,...,...
490,2021-08-25 00:00:00+00:00,Information Technology,Monolithic Power Systems,MPWR,380.841215,483.27,0.268954
491,2021-08-25 00:00:00+00:00,Information Technology,Gartner,IT,241.636900,307.19,0.271288
492,2021-08-25 00:00:00+00:00,Information Technology,Fortinet,FTNT,239.566100,306.03,0.277434
493,2021-08-25 00:00:00+00:00,Information Technology,Paycom,PAYC,378.839800,484.13,0.277928


In [13]:
full = sim[["date","adjClose","ticker","gain","rolling"]].merge(sp500[["Symbol","Security","GICS Sector"]].rename(columns={"Symbol":"ticker"}),on="ticker")[["date","GICS Sector","Security","ticker","rolling","adjClose","gain"]]

In [18]:
recent.index.size

495

In [17]:
market.connect()
market.store("full",full)
market.store("recent",recent)
market.disconnect()